# ライブラリのインポート

In [54]:
import pandas as pd
import numpy as np
from numpy import matlib

# サンプル候補の生成

In [55]:
number_of_generating_samples = 10000  # 生成するサンプル数
desired_sum_of_components = 1 # 合計を指定する特徴量がある場合の、合計の値。例えば、この値を 100 にすれば、合計を 100 にできます

setting_of_generation = pd.read_csv('setting_of_generation.csv', index_col=0, header=0)

# 0 から 1 の間の一様乱数でサンプル生成
x_generated = np.random.rand(number_of_generating_samples, setting_of_generation.shape[1])

# 上限・下限の設定
x_upper = setting_of_generation.iloc[0, :]  # 上限値
x_lower = setting_of_generation.iloc[1, :]  # 下限値
x_generated = x_generated * (x_upper.values - x_lower.values) + x_lower.values  # 上限値から下限値までの間に変換

# 合計を desired_sum_of_components にする特徴量がある場合
if setting_of_generation.iloc[2, :].sum() != 0:
    for group_number in range(1, int(setting_of_generation.iloc[2, :].max()) + 1):
        variable_numbers = np.where(setting_of_generation.iloc[2, :] == group_number)[0]
        actual_sum_of_components = x_generated[:, variable_numbers].sum(axis=1)
        actual_sum_of_components_converted = np.matlib.repmat(np.reshape(actual_sum_of_components, (x_generated.shape[0], 1)) , 1, len(variable_numbers))
        x_generated[:, variable_numbers] = x_generated[:, variable_numbers] / actual_sum_of_components_converted * desired_sum_of_components
        deleting_sample_numbers, _ = np.where(x_generated > x_upper.values)
        x_generated = np.delete(x_generated, deleting_sample_numbers, axis=0)
        deleting_sample_numbers, _ = np.where(x_generated < x_lower.values)
        x_generated = np.delete(x_generated, deleting_sample_numbers, axis=0)

# 数値の丸め込みをする場合
if setting_of_generation.shape[0] >= 4:
    for variable_number in range(x_generated.shape[1]):
        x_generated[:, variable_number] = np.round(x_generated[:, variable_number], int(setting_of_generation.iloc[3, variable_number]))

# 保存
x_generated = pd.DataFrame(x_generated, columns=setting_of_generation.columns)
x_generated.to_csv('generated_samples.csv')  # 生成した

# D最適基準によるサンプル候補の選定

In [56]:
number_of_selecting_samples = 30  # 選択するサンプル数
number_of_random_searches = 1000  # ランダムにサンプルを選択して D 最適基準を計算する繰り返し回数

x_generated = pd.read_csv('generated_samples.csv', index_col=0, header=0)
autoscaled_x_generated = (x_generated - x_generated.mean()) / x_generated.std()

# 実験条件の候補のインデックスの作成
all_indexes = list(range(x_generated.shape[0]))

# D 最適基準に基づくサンプル選択
np.random.seed(11) # 乱数を生成するためのシードを固定
for random_search_number in range(number_of_random_searches):
    # 1. ランダムに候補を選択
    new_selected_indexes = np.random.choice(all_indexes, number_of_selecting_samples, replace=False)
    new_selected_samples = autoscaled_x_generated.iloc[new_selected_indexes, :]
    # 2. D 最適基準を計算
    xt_x = np.dot(new_selected_samples.T, new_selected_samples)
    d_optimal_value = np.linalg.det(xt_x) 
    # 3. D 最適基準が前回までの最大値を上回ったら、選択された候補を更新
    if random_search_number == 0:
        best_d_optimal_value = d_optimal_value.copy()
        selected_sample_indexes = new_selected_indexes.copy()
    else:
        if best_d_optimal_value < d_optimal_value:
            best_d_optimal_value = d_optimal_value.copy()
            selected_sample_indexes = new_selected_indexes.copy()
selected_sample_indexes = list(selected_sample_indexes) # リスト型に変換

# 選択されたサンプル、選択されなかったサンプル
selected_samples = x_generated.iloc[selected_sample_indexes, :]  # 選択されたサンプル
remaining_indexes = np.delete(all_indexes, selected_sample_indexes)  # 選択されなかったサンプルのインデックス
remaining_samples = x_generated.iloc[remaining_indexes, :]  # 選択されなかったサンプル

# 保存
selected_samples.to_csv('selected_samples.csv')  # 選択されたサンプルを csv ファイルに保存。同じ名前のファイルがあるときは上書きされるため注意
remaining_samples.to_csv('remaining_samples.csv')  # 選択されなかったサンプルを csv ファイルに保存。同じ名前のファイルがあるときは上書きされるため注意

print(selected_samples.corr()) # 相関行列の確認

                raw material 1  raw material 2  raw material 3  pressure  \
raw material 1        1.000000       -1.000000        0.173479  0.060329   
raw material 2       -1.000000        1.000000       -0.173479 -0.060329   
raw material 3        0.173479       -0.173479        1.000000 -0.058115   
pressure              0.060329       -0.060329       -0.058115  1.000000   
time                  0.289715       -0.289715       -0.120124  0.201498   

                    time  
raw material 1  0.289715  
raw material 2 -0.289715  
raw material 3 -0.120124  
pressure        0.201498  
time            1.000000  
